In [25]:
import pandas as pd
csv_path = r"https://docs.google.com/spreadsheets/d/1EvIFLGpsKvuOgdX6F9MYlGl9rCLjbKHPlE8srBDMwYo/edit?usp=sharing"

# Download file from Google Sheets
url = csv_path.replace('/edit?usp=sharing', '/export?format=csv')
df = pd.read_csv(url)


Exp 1. "미개봉" 단어가 들어가 있다면, 해당 제품은 미개봉 제품일 것이다.

In [26]:
false_positive = df[(df['title'].str.contains('미개봉') | df['content'].str.contains('미개봉')) & (df['unused'] == False)]
false_positive

,id,date,writer,model,screen_size,chip,core,ram,ssd,apple_care_plus,unused,price,price for statistics,platform,url,title,content
207,211,2023-04-17,tyui363,Macbook Air,13,M1,"8,7",8GB,256GB,False,False,"850,000","850,000",중고나라,https://cafe.naver.com/joonggonara/984752629,맥북 에어 m1 2020 스페이스 그레이 256g 메모리 8g +가방 팝니다,👆 중고나라에도 앱이 있다는 것을 아시나요? 위에 다운로드 링크를 통해 중고나라 앱...
281,285,2023-04-21,stonebellk,Macbook Pro,16,M1Max,"10,24",64GB,1TB,True,False,"3,200,000","3,100,000",중고나라,https://cafe.naver.com/joonggonara/985530100,맥북프로 M1 MAX 16인치 (64GB / 1TB) 실버를 팝니다﻿.,1. 제품명/브랜드 : 맥북프로 M1 MAX 16인치 RAM : 64GB / SSD...
327,331,2023-04-24,낑깡 맛있엉,Mac Mini,0,M1,"8,8",16GB,256GB,False,False,"700,000","700,000",중고나라,https://cafe.naver.com/joonggonara/985993261,M1 맥미니 CTO 16g 256gb,1. 제품명/브랜드: M1 맥미니 CTO 16g 256gb 2. 상품 상태(미개봉/...
434,440,2023-05-02,bossmong79,Macbook Pro,14,M1Pro,"8,14",16GB,512GB,False,False,"1,000,000","1,000,000",중고나라,https://cafe.naver.com/joonggonara/987609935,2021 맥북 프로 14인치 M1 pro 기본형 판매 (램16 / 512ssd / 스그),👆 중고나라에도 앱이 있다는 것을 아시나요? 위에 다운로드 링크를 통해 중고나라 앱...


  - 4건의 false_positive가 발견되었지만, 이전 자동화 거치기 전 데이터를 포함한다면 더 많은 false_positive가 발견될 것이다.

 Exp 1.1 false_positive 제품들은 ChatGPT로부터 구별이 가능할 것이다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain import PromptTemplate
import os

unused_prompt = PromptTemplate(
    input_variables=["title", "content"],
    template=open("./prompts/model_screen_classify.txt", "r").read()
)

llm = ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-3.5-turbo")
chain_unused_classify: LLMChain = LLMChain(llm=llm, prompt=unused_prompt)